In [1]:
import sys

from qetu_sim import *

import numpy as np
import scipy.linalg
import itertools
import matplotlib.pyplot as plt

from qiskit import *
from qiskit.transpiler import Layout
from qiskit.quantum_info.operators import Operator
from qiskit.circuit.library import Permutation
from qiskit.visualization import *
from qiskit_aer import AerSimulator

In [2]:
u = 1
t = 1
delta_t = 1
n = 1
num_sites = 4

In [3]:
H_ref = ref_fh_hamiltonian(u=u, t=t, WMI_qubit_layout=True, include_aux=True)
λ, v = np.linalg.eigh(H_ref)
ground_state_energy = λ[0]
ground_state_vector = v[:,0]

In [4]:
noise_model = wmi_grid_noise_model()
sim_ideal = AerSimulator()
sim_noise = AerSimulator(noise_model=noise_model)

degree_list = range(30,31,2)
trotter_steps_list = range(3,4)
energy_estimation_list = []

for degree in degree_list:
    E_min, E_mu_m, E_mu_p, E_max = calculate_qsp_params(u, t)
    qsp = QSPPhase()
    phi_seq_su2 = qsp.cvx_qsp_heaviside(
        degree, 
        E_min,
        E_mu_m, 
        E_mu_p, 
        E_max
    )
    phi_vec = convert_Zrot_to_Xrot(phi_seq_su2)
    for trotter_steps in trotter_steps_list:
        print("Degree: " + str(degree) + "\t Trotter steps: " + str(trotter_steps))
        print("-------------------------")
        # Construct quantum circuit
        QETU_circ = construct_QETU_circ(u, t, trotter_steps, phi_vec)
        QETU_circ_WMI = transpile_QETU_to_WMI(QETU_circ)
        QETU_circ_WMI = add_pswap_labels(QETU_circ_WMI)
        QETU_circ_WMI = add_sy_labels(QETU_circ_WMI)
        QETU_circ_WMI = transpile(QETU_circ_WMI, sim_ideal, basis_gates=['pswap', 'cp', 'rz', 'sx', 'sy', 'x', 'y', 'unitary'])
        # prepare ground state
        initial_state = Statevector.from_label("+01+0-10-")
        circuit = QuantumCircuit(9, 1)
        circuit.initialize(initial_state)
        circuit.compose(QETU_circ_WMI, inplace=True)
        circuit.measure(4, 0)
        circuit.save_statevector()
        result = sim_ideal.run(transpile(circuit, sim_ideal)).result()
        psi = result.get_statevector(circuit)
        final_state = psi.data.real
        final_state_norm = final_state / np.linalg.norm(final_state, 2)
        overlap = abs(np.vdot(final_state_norm, ground_state_vector))**2
        print("overlap: " + str(overlap))
        # estimate ground state
        E0_meas = estimate_ground_state_energy(final_state_norm, u=1, t=1, num_shots=1_000)
        energy_estimation_list.append(E0_meas)
        print("E0_meas: " + str(E0_meas))
        print("|E0 - E0_meas|: " + str(abs(ground_state_energy - E0_meas)))


starting matlab engine..
Degree: 30	 Trotter steps: 3
-------------------------
overlap: 0.989866727634288
E0_meas: -22.185499999999998
|E0 - E0_meas|: 0.4584918268267799
